Just run these commands on terminal one by one for the required installations

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/project_codes/wsd

/content/drive/MyDrive/project_codes/wsd


In [4]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     - 541 kB 1.4 MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2330392 sha256=76f25dae518f35e90616cd8b95115b18698561646b82572d4433007e23903af3
  Stored in directory: /tmp/pip-ephem-wheel-cache-qnhc0r12/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7047e4e
Successfully built kenlm


In [5]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=2bd417c6d3f16db4b236927c862003232e15f2794280298aa7700890e264c07d
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [6]:
%cd /content/drive/MyDrive/project_codes/undreamt/undreamt

/content/drive/MyDrive/project_codes/undreamt/undreamt


In [13]:
%cd undreamt

/content/drive/My Drive/project_codes/undreamt/undreamt/undreamt


In [14]:
cd ..


/content/drive/MyDrive/project_codes/undreamt/undreamt


In [15]:
!cat /content/drive/MyDrive/project_codes/NLP/Master_Dataset/Hindi/eng_40000.txt | python translate.py --src_emb /content/drive/MyDrive/project_codes/Embeddings/en_40000.txt.vec \
                                           --tgt_emb /content/drive/MyDrive/project_codes/Embeddings/hi_40000.txt.vec \
                                           --sense_emb  /content/drive/MyDrive/project_codes/sense_embedding/sensegram/model/eng_40000.txt.cbow1-size300-window5-iter5-mincount10-bigramsFalse.word_vectors \
                                           --emb_dim 300\
                                           --lm /content/drive/MyDrive/project_codes/LM/kenlm/build/english_40k_lm_output.arpa \
                                           --cuda 0 \
                                           --topk 30\
                                           --similarity_measure csls\
                                           --lex_scaling 1\
                                           --lm_scaling 0.1\
                                           --beam_size 10 > trans_out_file_csls2


usage: translate.py [-h] [--batch_size BATCH_SIZE] [--beam_size BEAM_SIZE]
                    [--encoding ENCODING] [-i INPUT] [-o OUTPUT]
                    model
translate.py: error: unrecognized arguments: --src_emb --tgt_emb /content/drive/MyDrive/project_codes/Embeddings/hi_40000.txt.vec --sense_emb /content/drive/MyDrive/project_codes/sense_embedding/sensegram/model/eng_40000.txt.cbow1-size300-window5-iter5-mincount10-bigramsFalse.word_vectors --emb_dim 300 --lm /content/drive/MyDrive/project_codes/LM/kenlm/build/english_40k_lm_output.arpa --cuda 0 --topk 30 --similarity_measure csls --lex_scaling 1 --lm_scaling 0.1


In [10]:
!pwd

/content/drive/My Drive/project_codes/undreamt/undreamt/undreamt


In [12]:
cd ..


/content/drive/My Drive/project_codes/undreamt/undreamt


In [ ]:
%cd
!os.listdir

/root
/bin/bash: os.listdir: command not found


In [ ]:
utils.bool_flag()

AttributeError: ignored

In [ ]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# Copyright (c) 2018-present, Yunsu Kim
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#


import argparse
import os
import sys

import kenlm
from torch import nn
import sys
from util import bool_flag, load_embeddings, normalize_embeddings
#from . import utils
from sensegram import SenseGram
from translator import Translator
import gensim

def is_zip(fname):
    fp = open(fname, "rb")
    magic = fp.read(2)
    fp.close()
    return magic == '\x1f\x8b'

def open_file(file_arg, allowNone=True):
    if not file_arg:
        if allowNone:
            return None
        else:
            raise IOError
    if isinstance(file_arg, str):
        if is_zip(file_arg):
            return gzip.GzipFile(file_arg)
        else:
            return open(file_arg)
    elif isinstance(file_arg, file):
        return file_arg

def load_input(filename, lowercase):
    input_stream = sys.stdin
    if filename is not None:
        input_stream = open_file(filename)
    input_sents = []
    for line in input_stream:
        if lowercase:
            line = line.lower()
        input_sents.append(line.rstrip().split())
    return input_sents

sys.argv = ['']

parser = argparse.ArgumentParser(description='Evaluation')

    # data
parser.add_argument('--input', type=str, default='/content/drive/MyDrive/autoencoder_train/model_lm/test.en2', help='Source corpus to translate (default: stdin)')
parser.add_argument('--output', type=str, default='/content/drive/MyDrive/autoencoder_train/model_lm/trans', help='File name where the output translations will be written (default: stdout)')
parser.add_argument('--input_lowercase', type=bool_flag, default=True, help='Lowercase the given input corpus')

    # embedding
parser.add_argument('--src_emb', type=str, default='/content/drive/MyDrive/autoencoder_train/model_lm/aSRC_MAPPED_IITB.EMB', help='Source embeddings')
parser.add_argument('--tgt_emb', type=str, default='/content/drive/MyDrive/autoencoder_train/model_lm/aTRG_MAPPED_IITB.EMB', help='Target embeddings')
parser.add_argument('--sense_emb', type=str, default='/content/drive/MyDrive/autoencoder_train/model_lm/en.txt.cbow1-size300-window5-iter5-mincount10-bigramsFalse.n200.clusters.minsize5-1000-sum-score-20.sense_vectors', help='Sense embeddings')
parser.add_argument('--emb_dim', type=int, default=300, help='Embedding dimension')
parser.add_argument('--normalize_embeddings', type=str, default='', help='Normalize embeddings before translation')

    # search
parser.add_argument('--max_vocab', type=int, default=100000, help='Maximum source/target vocabulary size considered in search')
parser.add_argument('--similarity_measure', type=str, default='csls', help='Similarity measure used for nearest neighbor search (cosine|csls)')
parser.add_argument('--csls_k', type=int, default=10, help='Number of nearest neighbors for computing hub penalty term in csls')
parser.add_argument('--unknown_words', type=str, default='copy', help='Method to deal with unknown source word (copy|unk)')

    # lm integration
parser.add_argument('--lm', type=str,default='/content/drive/MyDrive/autoencoder_train/model_lm/mod2.arpa', help='Target language model file')
parser.add_argument('--lm_scaling', type=float, default=0.1, help='Language model scaling')
parser.add_argument('--lex_scaling', type=float, default=1.0, help='Lexicon model scaling')
parser.add_argument('--beam_size', type=int, default=10, help='Beam size')
parser.add_argument('--topk', type=int, default=100, help='Number of nearest neighbors considered')
parser.add_argument('--emb_scaling', type=str, default='linear', help='(linear|sigmoid|softmax)')
parser.add_argument('--softmax_temparature', type=float, default=1.0, help='Temparature parameter for scaling softmax distribution')

    # others
parser.add_argument('--cuda', type=bool_flag, default=True, help='Run on GPU')

    # parse parameters
params = parser.parse_args()
params.src_lang = 'src'
params.tgt_lang = 'tgt'  # lang code can be arbitrary since we don't load dictionary files

    # check parameters
#print(params.src_emb) 
assert os.path.isfile(params.src_emb)
assert os.path.isfile(params.tgt_emb)

    # load input (to translate)
print("loading input data...", file=sys.stderr)
input_sents = load_input(params.input, params.input_lowercase)  # CHECK: vocab?

    # load embeddings
print("loading embeddings...", file=sys.stderr)
src_dico, _src_emb = load_embeddings(params, source=True)  # 'dico' = word2id mappings
src_emb = nn.Embedding(len(src_dico), params.emb_dim, sparse=True)
src_emb.weight.data.copy_(_src_emb)
    
tgt_dico, _tgt_emb = load_embeddings(params, source=False)
tgt_emb = nn.Embedding(len(tgt_dico), params.emb_dim, sparse=True)
tgt_emb.weight.data.copy_(_tgt_emb)

if params.cuda:
        src_emb.cuda()
        tgt_emb.cuda()
    
sense_emb = SenseGram.load_word2vec_format(params.sense_emb, binary=False)
senWord_emb = gensim.models.KeyedVectors.load_word2vec_format(params.src_emb, binary=False)
    # normalize embeddings
print("normalizing embeddings...", file=sys.stderr)
params.src_mean = normalize_embeddings(src_emb.weight.data, params.normalize_embeddings)
params.tgt_mean = normalize_embeddings(tgt_emb.weight.data, params.normalize_embeddings)
    



loading input data...
loading embeddings...
normalizing embeddings...


In [ ]:

lm = None
if params.lm is not None:
      print("loading LM...", file=sys.stderr)
      lm = kenlm.LanguageModel(params.lm)



loading LM...


In [ ]:
    # translate
print("translating...", file=sys.stderr)
translator = Translator(src_emb, tgt_emb, src_dico, tgt_dico, sense_emb,senWord_emb, params)
translator.corpus_translation(input_sents, lm)

translating...


NameError: ignored